In [1]:
import numpy as np
import pandas as pd
import pymysql

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
user_info = pd.DataFrame({'email':'0', 'pw':'1234', 'gender':'female', 'age':20,
                        'large_category':'소설,소설,시에세이', 'middle_category':'영미소설,판타지소설,한국시',
                        'selected_book_isbn':'123456789,234567891,345678912'}, index=[0])
book_df = pd.DataFrame({'title':'해리포터', 'author':'조앤K롤링', 
              'large_category':'소설,소설', 'middle_category':'영미소설,판타지소설', 
              'b_rank':20, 'rate':9.4, 'count':1324, 'isbn': 566465456545}, index=[0])
display(user_info)
display(book_df)

,email,pw,gender,age,large_category,middle_category,selected_book_isbn
0,0,1234,female,20,"소설,소설,시에세이","영미소설,판타지소설,한국시","123456789,234567891,345678912"


,title,author,large_category,middle_category,b_rank,rate,count,isbn
0,해리포터,조앤K롤링,"소설,소설","영미소설,판타지소설",20,9.4,1324,566465456545


In [29]:
class sortBook():
    def __init__(self, user_info):
        self.gender = user_info['gender'].values[0]
        self.age = user_info['age'].values[0]
        g1s = user_info['large_category'].values[0].split(',')
        g2s = user_info['middle_category'].values[0].split(',')
        self.genres = {g1:{'df':self.call_df(g1),
                           g2:{}} for g1,g2 in zip(g1s, g2s)}
    
    def connect_mysql(self, local=True, connect=True):
        if connect:
            if local:
                host, user, pw = 'localhost', 'root', '1234'
            else:
                host, user, pw = '192.168.0.176', 'pagepalette', 'pagepalette0528'
            self.db = pymysql.connect(host=host, port=3306, user=user, passwd=pw, db='pagepalette', 
                                      charset='utf8', cursorclass=pymysql.cursors.DictCursor)
            self.cursor = self.db.cursor()
        else:
            self.db.commit()
            self.db.close
    
    def call_df(self, g1):
        self.connect_mysql(local=False, connect=True)
        sql = 'SELECT * FROM gyobo_{}'.format(g1)
        self.cursor.execute(sql)
        df = pd.DataFrame(self.cursor.fetchall())
        self.connect_mysql(local=False, connect=False)
        
        df.drop_duplicates(subset=['title','author'], ignore_index=True, inplace=True)
        return df.dropna()
    
    def recomm1_rank(self, g1, g2, n=240, check=True):
        df = self.genres[g1]['df'].copy()
        
        try: idx = df[df['middle_category']==g2].index.values[0]
        except:
            check = False
            df = pd.concat([df, pd.DataFrame({'middle_category':[g2]})], ignore_index=True)
            idx = df[df['middle_category']==g2].index.values

        cvect = CountVectorizer(ngram_range=(1,3))
        cvect_g2 = cvect.fit_transform(df['middle_category'])
        csim_g2 = cosine_similarity(cvect_g2, cvect_g2).argsort()[:,::-1]
        
        sim_idx = csim_g2[idx, :n].reshape(-1)
        sim_idx = sim_idx if check else sim_idx[sim_idx != idx]
        self.genres[g1][g2]['df'] = df.iloc[sim_idx].sort_values(['b_rank','rate'], ascending=[True, False])
    
    def __call__(self):
        select_dfs, i = [], 1
        g1_n, g1_c = 480//len(self.genres.keys()), 12//len(self.genres.keys())
        for g1, g2s in self.genres.items():
            g2_n, g2_c = g1_n//(len(g2s.keys())-1), g1_c//(len(g2s.keys())-1)
            for g2 in g2s.keys():
                if g2=='df': continue
                self.recomm1_rank(g1, g2, g2_n)
                df = self.genres[g1][g2]['df'].iloc[:g2_c,:]
                df['order'] = i
                i += 1
                select_dfs.append(df)
        return pd.concat(select_dfs, axis=0)
    
check = sortBook(user_info)
check()

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [3]:
class recommBook():
    def __init__(self, user_info):
        self.gender = user_info['gender'].values[0]
        self.age = user_info['age'].values[0]
        
        sort = sortBook(user_info)
        sort_dfs = sort()
        g1s = user_info['large_category'].values[0].split(',')
        g2s = user_info['middle_category'].values[0].split(',')
        self.selected_n = len(g2s)
        select_books = user_info['selected_book_isbn'].values[0].split(',')
        if self.selected_n==1:
            selected_books = [selected_books]
        elif self.selected_n==2:
            selected_books = [selected_books[:1], selected_books[2:]]
        elif self.selected_n==3:
            selected_books = [selected_books[:1], selected_books[2], selected_books[3]]
        self.genres = {g1:{g2:{'isbn':isbn,
                               'df':sort_dfs[g1][g2]['df']}} for g1,g2,isbn in zip(g1s, g2s, select_books)}
        self.df_ga = self.make_df_ga()
    
    def connect_mysql(self, local=True, connect=True):
        if connect:
            if local:
                host, user, pw = 'localhost', 'root', '1234'
            else:
                host, user, pw = '192.168.0.176', 'pagepalette', 'pagepalette0528'
            self.db = pymysql.connect(host=host, port=3306, user=user, passwd=pw, db='pagepalette', 
                                      charset='utf8', cursorclass=pymysql.cursors.DictCursor)
            self.cursor = self.db.cursor()
        else:
            self.db.commit()
            self.db.close
    
    def make_df_ga(self):
        self.connect_mysql(local=False, connect=True)
        sql = 'SELECT * FROM {}_{}'.format(self.age, self.gender)
        self.cursor.execute(sql)
        df = pd.DataFrame(self.cursor.fetchall())
        self.connect_mysql(local=False, connect=False)
        return df
    
    def recomm2_rate(self, g1, g2, selected_isbn):
        df = self.genres[g1][g2]['df'].copy().reset_index(drop=True)
        n = len(df)//2
        
        cvect = CountVectorizer(ngram_range=(1,3))
        cvect_g2 = cvect.fit_transform(df['middle_category'])
        csim_g2 = cosine_similarity(cvect_g2, cvect_g2).argsort()[:,::-1]
        
        idx = df[df['isbn'] == selected_isbn].index.values
        sim_idx = csim_g2[idx, :n].reshape(-1)
        sim_idx = sim_idx[sim_idx != idx]
        return df.iloc[sim_idx].sort_values(['b_rank', 'rate'], ascending=[True, False])
    
    def recomm2_ga(self, g1, g2, selected_isbn):
        df = self.recomm2_rate(g1, g2, selected_isbn)
        df_ga = pd.merge(self.genres[g1][g2]['df'], self.df_ga, how='inner', 
                         left_on='isbn', right_on='agender_isbn')[['agender_rank','agender_isbn']]
        
        bins, labels = [i for i in range(0,210,10)], [i for i in range(-200, 0, 10)]
        df_ga['ga_new_rank'] = pd.cut(df_ga['{}_{}_rank'.format(self.age, self.gender)], bins=bins, labels=labels)
        new_df = pd.merge(df, df_ga, how='left', left_on='isbn', right_on='agender_isbn').drop(columns=['agender_isbn'])
        new_df['ga_new_rank'] = new_df['ga_new_rank'].astype({'ga_new_rank':float})
        new_df['ga_new_rank'] = new_df['ga_new_rank'].fillna(0)
        new_df['new_rank'] = new_df['b_rank'] + new_df['ga_new_rank']
        return new_df.sort_values(['new_rank','rate'], ascending=[True, False])
    
    def __call__(self):
        result_li = []
        for g1, g2s in self.genres.items():
            for g2 in g2s.keys():
                if g2=='df': continue
                for i, selected_isbn in enumerate(self.genres[g1][g2]['isbn']):
                    author = self.genres[g1][g2]['df'].loc[self.genres[g1][g2]['df']['isbn']==selected_isbn, 'author']
                    author_df = self.genres[g1][g2]['df'][self.genres[g1][g2]['df']['author']=='author']
                    df = pd.concat([self.recomm2_ga(g1, g2, selected_isbn).iloc[:50,:], author_df], axis=0)
                    df.drop_duplicates(subset=['title','author'], ignore_index=True, inplace=True)

                    recomm_n = 2 if self.selected_n<=2 else 1
                    if self.selected_n==3 and i==0: recomm_n = 2
                    idx = np.random.randint(len(df), size=recomm_n)
                    result_li.append(df.loc[idx])
        result_df = pd.concat(result_li, axis=0)
        
        print('추천하는 책입니다.')
        display(result_df)
        
myrecomm = recommBook(user_info)
myrecomm()

In [5]:
class recommBook():
    def __init__(self, gender, age, genres):
        self.gender, self.age = gender, age
        self.ga = 2*(age//10)-2 if gender=='female' else 2*(age//10)-1
        self.df_ga = self.make_df_ga()
        self.genres = {g1:{g2:{} for g2 in g2s} for g1,g2s in genres.items()}
        self.selected_n = 0
        for g1 in genres.keys():
            self.genres[g1]['df'] = self.call_df(g1)
            self.selected_n += len(genres[g1])
        print('user info: {}, {}, {}'.format(gender, age, genres))
        
    def call_df(sefl, g1):
        df = pd.read_csv('./best_books/gyobo_{}.csv'.format(g1))
        df.drop_duplicates(subset=['title','author'], ignore_index=True, inplace=True)
        return df.dropna()
    
    def make_df_ga(self):
        df = pd.read_csv('./age_gender_popular_book/{}_{}.csv'.format(self.age, self.gender), 
                            encoding='cp949', sep='\t')

        df = df.drop(df.iloc[[0, 1, 2]].index.values).reset_index(drop=True)
        df = df['제공기관 : 국립중앙도서관'].str.split(''',"''', expand=True)
        df.columns = ['rank', 'title', 'author', 'publisher', 'pub_year','series', 'isbn', 'loan_cnt']
        for col in ['title','author','isbn']:
            df[col] = df[col].str.replace('"','')
        
        df = df.astype({'rank':float,'isbn':float})
        df.rename(columns = {'rank':'{}_{}_rank'.format(self.age, self.gender),'isbn':'library_isbn'}, inplace=True)
        return df[['{}_{}_rank'.format(self.age, self.gender), 'library_isbn']]
    
    def recomm1_rank(self, g1, g2, n=240, check=True):
        df = self.genres[g1]['df'].copy()
        
        try: idx = df[df['genre_2']==g2].index.values[0]
        except:
            check = False
            df = pd.concat([df, pd.DataFrame({'genre_2':[g2]})], ignore_index=True)
            idx = df[df['genre_2']==g2].index.values

        cvect = CountVectorizer(ngram_range=(1,3))
        cvect_g2 = cvect.fit_transform(df['genre_2'])
        csim_g2 = cosine_similarity(cvect_g2, cvect_g2).argsort()[:,::-1]
        
        sim_idx = csim_g2[idx, :n].reshape(-1)
        sim_idx = sim_idx if check else sim_idx[sim_idx != idx]
        self.genres[g1][g2]['df'] = df.iloc[sim_idx].sort_values(['brank','rate'], ascending=[True, False])
    
    def recomm1(self):
#         select_dfs = []
        g1_n, g1_c = 480//len(self.genres.keys()), 12//len(self.genres.keys())
        for g1, g2s in self.genres.items():
            g2_n, g2_c = g1_n//(len(g2s.keys())-1), g1_c//(len(g2s.keys())-1)
            for g2 in g2s.keys():
                if g2=='df': continue
                self.recomm1_rank(g1, g2, g2_n)
                df = self.genres[g1][g2]['df'].iloc[:g2_c,:]
#                 select_dfs.append(df)
                display(df)
                if self.selected_n==1:
                    idxs = map(int, input('{}_{} 마음에 드는 책의 idx 두 개 입력:'.format(g1,g2)).split())
                    self.genres[g1][g2]['isbn'] = [df.loc[idx,'isbn'] for idx in idxs]
                else:
                    idx = int(input('{}_{} 마음에 드는 책의 idx:'.format(g1, g2)))
                    self.genres[g1][g2]['isbn'] = [df.loc[idx,'isbn']]
#         pd.concat(select_dfs, axis=0)
    
    def recomm2_rate(self, g1, g2, selected_isbn):
        df = self.genres[g1][g2]['df'].copy().reset_index(drop=True)
        n = len(df)//2
        
        cvect = CountVectorizer(ngram_range=(1,3))
        cvect_g2 = cvect.fit_transform(df['genre_2'])
        csim_g2 = cosine_similarity(cvect_g2, cvect_g2).argsort()[:,::-1]
        
        idx = df[df['isbn'] == selected_isbn].index.values
        sim_idx = csim_g2[idx, :n].reshape(-1)
        sim_idx = sim_idx[sim_idx != idx]
        return df.iloc[sim_idx].sort_values(['brank', 'rate'], ascending=[True, False])
    
    def recomm2_ga(self, g1, g2, selected_isbn):
        df = self.recomm2_rate(g1, g2, selected_isbn)
        df_ga = pd.merge(self.genres[g1][g2]['df'], self.df_ga, how='inner', 
                         left_on='isbn', right_on='library_isbn')[['{}_{}_rank'.format(self.age, self.gender),'library_isbn']]
        
        bins, labels = [i for i in range(0,210,10)], [i for i in range(-200, 0, 10)]
        df_ga['ga_new_rank'] = pd.cut(df_ga['{}_{}_rank'.format(self.age, self.gender)], bins=bins, labels=labels)
        new_df = pd.merge(df, df_ga, how='left', left_on='isbn', right_on='library_isbn').drop(columns=['library_isbn'])
        new_df['ga_new_rank'] = new_df['ga_new_rank'].astype({'ga_new_rank':float})
        new_df['ga_new_rank'] = new_df['ga_new_rank'].fillna(0)
        new_df['new_rank'] = new_df['brank'] + new_df['ga_new_rank']
        return new_df.sort_values(['new_rank','rate'], ascending=[True, False])
    
    def recomm2(self):
        result_li = []
        for g1, g2s in self.genres.items():
            for g2 in g2s.keys():
                if g2=='df': continue
                for selected_isbn in self.genres[g1][g2]['isbn']:
                    author = self.genres[g1][g2]['df'].loc[self.genres[g1][g2]['df']['isbn']==selected_isbn, 'author']
                    author_df = self.genres[g1][g2]['df'][self.genres[g1][g2]['df']['author']=='author']
                    df = pd.concat([self.recomm2_ga(g1, g2, selected_isbn).iloc[:50,:], author_df], axis=0)
                    df.drop_duplicates(subset=['title','author'], ignore_index=True, inplace=True)

                    recomm_n = 2 if self.selected_n<=2 else 1
                    idx = np.random.randint(len(df), size=recomm_n)
                    result_li.append(df.loc[idx])
        result_df = pd.concat(result_li, axis=0)
        
        print('추천하는 책입니다.')
        display(result_df)
        
gender, age = 'male', 20
genres = {
#           '소설': ['영미소설','세계문학전집'],
          '시에세이': ['영미에세이']
#           '인문': ['인문학일반','심리학']
         } # g1:[g2_1, g2_2]
myrecomm = recommBook(gender, age, genres)
# myrecomm.recomm1('소설', '한국소설').iloc[:2,:]
myrecomm.recomm1()
myrecomm.recomm2()

user info: male, 20, {'시에세이': ['영미에세이']}


,Unnamed: 0,title,author,genre_1,genre_2,brank,rate,count,isbn
843,915.0,전쟁 같은 맛,그레이스 M. 조,"시/에세이,","나라별 에세이, 영미에세이,",15.0,10.00,1.0,9.791169e+12
2,2.0,하늘과 바람과 별과 시,윤동주100년포럼,"시/에세이,","한국시, 현대시,",28.0,9.93,112.0,9.791158e+12
874,946.0,불안,알랭 드 보통,"시/에세이,","나라별 에세이, 영미에세이,",47.0,9.44,343.0,9.788957e+12
781,851.0,"위로를 주는 빵집, 오렌지 베이커리",키티 테이트 외,"시/에세이, 요리,","나라별 에세이, 영미에세이, 요리에세이,",51.0,10.00,1.0,9.791156e+12
792,862.0,월든(완결판),헨리 데이빗 소로우,"시/에세이,","나라별 에세이, 영미에세이,",62.0,9.42,336.0,9.788957e+12
796,866.0,삶으로 다시 떠오르기,에크하르트 톨레,"시/에세이, 시/에세이,","테마에세이, 명상/치유에세이, 나라별 에세이, 영미에세이,",66.0,9.65,256.0,9.791195e+12
817,888.0,소년과 두더지와 여우와 말,찰리 맥커시,"시/에세이, 시/에세이,","테마에세이, 그림에세이, 나라별 에세이, 영미에세이,",88.0,9.81,146.0,9.788997e+12
825,897.0,H마트에서 울다,미셸 자우너,"시/에세이,","나라별 에세이, 영미에세이,",97.0,9.74,179.0,9.788955e+12
734,801.0,사랑은 찾는 게 아니라 만드는 것이다,정이수,"시/에세이,","나라별 에세이, 한국에세이,",101.0,9.58,12.0,9.791193e+12
735,802.0,나의 봄날인 너에게,여수언니(정혜영),"시/에세이,","나라별 에세이, 한국에세이,",102.0,9.87,43.0,9.791131e+12


시에세이_영미에세이 마음에 드는 책의 idx 두 개 입력:874 781
추천하는 책입니다.


,Unnamed: 0,title,author,genre_1,genre_2,brank,rate,count,isbn,20_male_rank,ga_new_rank,new_rank
45,470.0,태도에 관하여,임경선,"시/에세이,","나라별 에세이, 한국에세이,",518.0,9.47,152.0,9.791160e+12,NaN,0.0,518.0
26,692.0,얀 마텔 101통의 문학 편지,얀 마텔,"시/에세이,","나라별 에세이, 영미에세이,",342.0,9.63,24.0,9.791160e+12,NaN,0.0,342.0
49,411.0,"나는 간호사, 사람입니다",김현아,"시/에세이,","나라별 에세이, 한국에세이,",558.0,10.00,1.0,9.791198e+12,NaN,0.0,558.0
5,897.0,H마트에서 울다,미셸 자우너,"시/에세이,","나라별 에세이, 영미에세이,",97.0,9.74,179.0,9.788955e+12,NaN,0.0,97.0
